In [1]:
import numpy as np

%precision 4

u'%.4f'

In [2]:
def p2r(m, a):
    x = m * np.cos(a*np.pi/180.0)
    y = m * np.sin(a*np.pi/180.0)
    return x + 1j * y

def r2p(z):
    m = np.absolute(z)
    a = np.angle(z) * 180.0 / np.pi
    return (m, a)

# Fluxo de carga em sistemas radiais

Para o sistema mostrado na Figura abaixo a impedância da linha é dada por:
$$ z = 0.3 + j0.6~\Omega/milha $$

E as cargas são conectadas nas barras 2 e 3 são dadas por:

$$ S_2 = 1.500 + j750 ~ kVA $$

$$ S_3 = 900 + j500 kVA $$

In [3]:
z = 0.3 + 0.6j
z

(0.3+0.6j)

In [4]:
s2 = 1500e3 + 750e3j
s2

(1500000+750000j)

In [5]:
s3 = 900e3 + 500e3j
s3

(900000+500000j)

A tensão nominal, de fase, na barra da subestação é dada por: 

In [6]:
v1 = 7.2e3
v1

7200.0000

<img src='Figuras/fig_066.png' width=400/>

calculando as impedancias do sistema:

In [7]:
l12 = 3e3
l23 = 4e3

In [8]:
z12 = z * l12 / 5280.0
z12

(0.17045454545454544+0.3409090909090909j)

In [9]:
z23 = z * l23 / 5280.0
z23

(0.22727272727272727+0.45454545454545453j)

## Varredura inversa para i=1:

cálculo das correntes

$$ I_i = \left( \frac{S_i}{V_i} \right)^{*} $$

Cálculo da corrente consumida na barra 3 

In [10]:
i3 = np.conj(s3 / v1)
r2p(i3)

(142.9949, -29.0546)

Cálculo da corrente no trecho 2-3

In [11]:
i23 = i3

Cálculo da corrente consumida na barra 2

In [12]:
i2 = np.conj(s2 / v1)
r2p(i2)

(232.9237, -26.5651)

Cálculo da corrente no trecho 1-2

In [13]:
i12 = i23 + i2
r2p(i12)

(375.8350, -27.5120)

## Varredura direta para i=1:

Cálculo das tensões:

$$ V_{i+1} = V_{i} - Z_{i-(i+1)} \cdot I_{i-(i+1)} $$

Cálculo da tensão na barra 2: 

In [14]:
v2 = v1 - z12 * i12
r2p(v2)

(7084.4947, -0.6797)

Cálculo da tensão na barra 3:

In [15]:
v3 = v2 - z23 * i23
r2p(v3)

(7025.1350, -1.0202)

Caculadas as tensões em todas as barras do alimentador, encerra-se a fase de varredura direta e inicia-se uma nova fase de varredura inversa, i=2, onde as correntes são calculadas com os novos valores de tensão encontrados na varredura direta anterior.

O processo é repetido até que a precisão necessária seja alcançada.

## Exemplo completo de varredura direta inversa

In [16]:
from terminaltables import AsciiTable

tl = [['Iteracao', 'Mag. v2', 'Ang. v2', 'Mag. v3', 'Ang. v3']]


# valores iniciais das tensoes
v2 = v3 = v1 = 7.2e3
e = 1e3
c = 0

while e > 0.0001 and c < 50:
    
    # registro das tensoes e angulos em cada iteracao
    tl.append([str(c), # iteracao
               str(r2p(v2)[0]),  # modulo da tensao na barra 2
               str(r2p(v2)[1]),  # angulo da tensao na barra 2
               str(r2p(v3)[0]),  # modulo da tensao na barra 3
               str(r2p(v3)[1])], # angulo da tensao na barra 3
             )
    
    # VARREDURA INVERSA
    
    # calculo da carrente na carga 3
    i3 = np.conj(s3 / v3)
    # calculo da corrente na linha 2-3
    i23 = i3
    # calculo da corrente na carga 2
    i2 = np.conj(s2 / v2)
    # calculo da corrente na linha 1-2
    i12 = i2 + i23
    
    # VARREDURA DIRETA
    
    # calculo da tensao na barra 2
    v2_ = v1 - z12 * i12
    
    # calculo da tensao na barra 3
    v3_ = v2 - z23 * i23
    
    # Calculo da precisao
    e = max(np.abs(v2_ - v2), np.abs(v3_ - v3))
    
    # atualizacao dos valores das tensoes
    v2 = v2_
    v3 = v3_
    
    # incremento do contador
    c += 1

    
# exibicao da evolucao da tensao nas barras 2 e 3
t = AsciiTable(tl)
print t.table

+----------+---------------+-----------------+---------------+-----------------+
| Iteracao | Mag. v2       | Ang. v2         | Mag. v3       | Ang. v3         |
+----------+---------------+-----------------+---------------+-----------------+
| 0        | 7200.0        | 0.0             | 7200.0        | 0.0             |
| 1        | 7084.49473522 | -0.67971738989  | 7140.14317132 | -0.329288286677 |
| 2        | 7082.14518614 | -0.679984848418 | 7024.39548663 | -1.02024851581  |
| 3        | 7080.99624447 | -0.679803600318 | 7020.54622822 | -1.02026946216  |
| 4        | 7080.95930738 | -0.68001514433  | 7019.36345308 | -1.02033038084  |
| 5        | 7080.95089285 | -0.680057534643 | 7019.31626314 | -1.02060292009  |
| 6        | 7080.95030667 | -0.68005754242  | 7019.30726611 | -1.02064601145  |
| 7        | 7080.95021777 | -0.680057621265 | 7019.30656943 | -1.02064611045  |
+----------+---------------+-----------------+---------------+-----------------+
